In [9]:
!pip install pygame pandas scikit-learn spotipy

  Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl.metadata (13 kB)
Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl (10.6 MB)


In [10]:
import os
import random
import pygame
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def load_music(directory):
    """Loads music files from the specified directory."""
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.mp3')]

def play_music(track):
    """Plays the selected music track."""
    pygame.mixer.init()
    pygame.mixer.music.load(track)
    pygame.mixer.music.play()
    print(f'Now Playing: {os.path.basename(track)}')

def fetch_trending_tracks():
    """Fetches real-time trending tracks using Spotify API."""
    client_id = "your_spotify_client_id"
    client_secret = "your_spotify_client_secret"
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))
    results = sp.playlist_tracks("37i9dQZF1DXcBWIGoYBM5M")  # Example: Global Top 50
    
    trending_tracks = []
    for item in results['items']:
        track = item['track']['name'] + ' - ' + item['track']['artists'][0]['name']
        trending_tracks.append(track)
    
    return trending_tracks

def recommend_songs(user_history, all_songs):
    """Recommends songs based on historical listening data."""
    if len(user_history) < 2:
        return random.sample(all_songs, min(len(all_songs), 5))
    
    df = pd.DataFrame({'song': all_songs, 'feature': [random.random() for _ in all_songs]})
    history_df = pd.DataFrame({'song': user_history, 'feature': [random.random() for _ in user_history]})
    
    model = NearestNeighbors(n_neighbors=5, metric='euclidean')
    model.fit(df[['feature']])
    
    _, indices = model.kneighbors(history_df[['feature']])
    recommended = df.iloc[indices.flatten()]['song'].tolist()
    return list(set(recommended) - set(user_history))

def main():
    music_directory = "./music"
    all_songs = load_music(music_directory)
    user_history = []
    
    while True:
        print("\nOptions: 1-Play Random, 2-Recommend Songs, 3-Trending Tracks, 4-Exit")
        choice = input("Choose an option: ")
        
        if choice == '1':
            track = random.choice(all_songs)
            play_music(track)
            user_history.append(track)
        elif choice == '2':
            recommendations = recommend_songs(user_history, all_songs)
            print("Recommended Songs:", recommendations)
        elif choice == '3':
            trending = fetch_trending_tracks()
            print("Trending Tracks:", trending)
        elif choice == '4':
            break
        else:
            print("Invalid choice, try again.")

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


FileNotFoundError: [WinError 3] The system cannot find the path specified: './music'